# 01_02_remove_duplicated_fits_file

## 필요한 모듈

이 프로젝트를 위해서는 아래의 모듈이 필요하다. 

> numpy, pandas, matplotlib, astropy, version_information

### 모듈 설치

1. 콘솔 창에서 모듈을 설치할 때는 아래와 같은 형식으로 입력하면 된다.

>pip install module_name==version

>conda install module_name=version

2. 주피터 노트북(코랩 포함)에 설치 할 때는 아래의 셀을 실행해서 실행되지 않은 모듈을 설치할 수 있다. (pip 기준) 만약 아나콘다 환경을 사용한다면 7행을 콘다 설치 명령어에 맞게 수정하면 된다.

In [131]:
#import sys
#!pip install astropy==5.2 photutils==1.6 #astroscrappy==1.1.1
#%pip install astroscrappy==1.1.1

In [132]:
import importlib, sys, subprocess
packages = "numpy, pandas, matplotlib, scipy, astropy, photutils, ccdproc, astroscrappy, version_information" # required modules
pkgs = packages.split(", ")
for pkg in pkgs :
    if not importlib.util.find_spec(pkg):
        #print(f"**** module {pkg} is not installed")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg, '-q'])
    else: 
        print(f"**** module {pkg} is installed")

%load_ext version_information
import time
now = time.strftime("%Y-%m-%d %H:%M:%S (%Z = GMT%z)")
print(f"This notebook was generated at {now} ")

vv = %version_information {packages}
for i, pkg in enumerate(vv.packages):
    print(f"{i} {pkg[0]:10s} {pkg[1]:s}")

**** module numpy is installed
**** module pandas is installed
**** module matplotlib is installed
**** module scipy is installed
**** module astropy is installed
**** module photutils is installed
**** module ccdproc is installed
**** module astroscrappy is installed
**** module version_information is installed
The version_information extension is already loaded. To reload it, use:
  %reload_ext version_information
This notebook was generated at 2024-10-01 23:25:38 (KST = GMT+0900) 
0 Python     3.12.3 64bit [GCC 11.2.0]
1 IPython    8.25.0
2 OS         Linux 6.8.0 45 generic x86_64 with glibc2.39
3 numpy      1.26.4
4 pandas     2.2.2
5 matplotlib 3.8.4
6 scipy      1.13.1
7 astropy    6.1.0
8 photutils  1.12.0
9 ccdproc    2.4.2
10 astroscrappy 1.2.0
11 version_information 1.0.4


### import modules

In [133]:
from glob import glob
from pathlib import Path, PosixPath, WindowsPath
import os
from astropy.time import Time
from datetime import datetime, timedelta
from astropy.io import fits
import shutil 

import ysfitsutilpy as yfu
#import ysphotutilpy as ypu
#import ysvisutilpy as yvu

import _astro_utilities
import _Python_utilities

In [134]:
#%%
#######################################################
BASEDIR = Path("/mnt/Rdata/ASTRO_data")  

PROJECDIR = BASEDIR / "C1-Variable"
TODODIR = PROJECDIR / "-_-_-_2016-_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2017-01_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2017-03_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2017-05_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2017-06_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2021-10_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2022-01_-_RiLA600_STX-16803_-_2bin"

# PROJECDIR = BASEDIR / "C2-Asteroid"
# TODODIR = PROJECDIR / "-_-_-_2022-_-_GSON300_STF-8300M_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2022-_-_RiLA600_STX-16803_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2022-_-_RiLA600_STX-16803_-_2bin"
# TODODIR = PROJECDIR / "-_-_-_2023-_-_GSON300_STF-8300M_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2023-_-_RiLA600_STX-16803_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2023-_-_RiLA600_STX-16803_-_2bin"

# PROJECDIR = BASEDIR / "C3-EXO"
# TODODIR = PROJECDIR / "-_-_-_2024-05_-_GSON300_STF-8300M_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2024-05_-_RiLA600_STX-16803_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin"
# TODODIR = PROJECDIR / "-_-_-_2024-06_-_RiLA600_STX-16803_-_2bin"

# PROJECDIR = BASEDIR / "C4-Spectra"
# TODODIR = PROJECDIR / "-_-_-_2024-05_TEC140_ASI183MMPro_-_1bin"

DOINGDIRs = sorted(_Python_utilities.getFullnameListOfsubDirs(TODODIR))
print ("DOINGDIRs: ", format(DOINGDIRs))
print ("len(DOINGDIRs): ", format(len(DOINGDIRs)))

try : 
    BDFDIR = [x for x in DOINGDIRs if "CAL-BDF" in str(x)]
    print ("BDFDIR: ", format(BDFDIR))
    MASTERDIR = Path(BDFDIR[0]) / _astro_utilities.master_dir
    if not MASTERDIR.exists():
        os.makedirs("{}".format(str(MASTERDIR)))
        print("{} is created...".format(str(MASTERDIR)))
    print ("MASTERDIR: ", format(MASTERDIR))
except : 
    pass

DOINGDIRs = sorted([x for x in DOINGDIRs if "_LIGHT_" in str(x)])
# print ("DOINGDIRs: ", format(DOINGDIRs))
# print ("len(DOINGDIRs): ", format(len(DOINGDIRs)))

# filter_str = 'BL'
# DOINGDIRs = [x for x in DOINGDIRs if filter_str in str(x)]
# remove = 'BIAS'
# DOINGDIRs = [x for x in DOINGDIRs if remove not in x]
# remove = 'DARK'
# DOINGDIRs = [x for x in DOINGDIRs if remove not in x]
# remove = 'FLAT'
# DOINGDIRs = [x for x in DOINGDIRs if remove not in x]
print ("DOINGDIRs: ", DOINGDIRs)
print ("len(DOINGDIRs): ", len(DOINGDIRs))

DOINGDIRs:  ['/mnt/Rdata/ASTRO_data/C1-Variable/-_-_-_2016-_-_RiLA600_STX-16803_-_2bin/BL-CAM_LIGHT_-_2016-10-03_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/ASTRO_data/C1-Variable/-_-_-_2016-_-_RiLA600_STX-16803_-_2bin/BL-CAM_LIGHT_-_2016-10-08_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/ASTRO_data/C1-Variable/-_-_-_2016-_-_RiLA600_STX-16803_-_2bin/BL-CAM_LIGHT_-_2016-10-09_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/ASTRO_data/C1-Variable/-_-_-_2016-_-_RiLA600_STX-16803_-_2bin/BL-CAM_LIGHT_-_2016-10-13_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/ASTRO_data/C1-Variable/-_-_-_2016-_-_RiLA600_STX-16803_-_2bin/BL-CAM_LIGHT_-_2016-11-08_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/ASTRO_data/C1-Variable/-_-_-_2016-_-_RiLA600_STX-16803_-_2bin/BL-CAM_LIGHT_-_2016-11-11_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/ASTRO_data/C1-Variable/-_-_-_2016-_-_RiLA600_STX-16803_-_2bin/BL-CAM_LIGHT_-_2016-11-15_-_RiLA600_STX-16803_-_2bin/', '/mnt/Rdata/ASTRO_data/C1-Variable/-_-_-_2016-_-_RiLA600_STX-16803_-_2bin/BL-

In [135]:
# DOINGDIRs[7]

In [136]:
for DOINGDIR in DOINGDIRs[:] : 
    DOINGDIR = Path(DOINGDIR)
    print(f"Starting: {str(DOINGDIR.parts[-1])}")
    if (DOINGDIR / f"reduced_DPhot_Mag12.5").exists():
        shutil.move(str(DOINGDIR / f"reduced_DPhot_Mag12.5"),
                        DOINGDIR / f"reduced_DPhot_Mag12.5_fw6")
    if (DOINGDIR / f"reduced_nightsky_DPhot_Mag12.5").exists() :
        shutil.move(str(DOINGDIR / f"reduced_nightsky_DPhot_Mag12.5"),
                        DOINGDIR / f"reduced_nightsky_DPhot_Mag12.5_fw6")
    if (DOINGDIR / f"reduced_DPhot_Mag11").exists():
        shutil.move(str(DOINGDIR / f"reduced_DPhot_Mag11"),
                        DOINGDIR / f"reduced_DPhot_Mag11_fw6")
    if (DOINGDIR / f"reduced_nightsky_DPhot_Mag11").exists() :
        shutil.move(str(DOINGDIR / f"reduced_nightsky_DPhot_Mag11"),
                        DOINGDIR / f"reduced_nightsky_DPhot_Mag11_fw6")

Starting: BL-CAM_LIGHT_-_2016-10-03_-_RiLA600_STX-16803_-_2bin
Starting: BL-CAM_LIGHT_-_2016-10-08_-_RiLA600_STX-16803_-_2bin
Starting: BL-CAM_LIGHT_-_2016-10-09_-_RiLA600_STX-16803_-_2bin
Starting: BL-CAM_LIGHT_-_2016-10-13_-_RiLA600_STX-16803_-_2bin
Starting: BL-CAM_LIGHT_-_2016-11-08_-_RiLA600_STX-16803_-_2bin
Starting: BL-CAM_LIGHT_-_2016-11-11_-_RiLA600_STX-16803_-_2bin
Starting: BL-CAM_LIGHT_-_2016-11-15_-_RiLA600_STX-16803_-_2bin
Starting: BL-CAM_LIGHT_-_2016-11-16_-_RiLA600_STX-16803_-_2bin
Starting: BL-CAM_LIGHT_-_2016-11-17_-_RiLA600_STX-16803_-_2bin
Starting: BL-CAM_LIGHT_-_2016-11-20_-_RiLA600_STX-16803_-_2bin
Starting: BL-CAM_LIGHT_-_2016-11-28_-_RiLA600_STX-16803_-_2bin
Starting: BL-CAM_LIGHT_-_2016-12-01_-_RiLA600_STX-16803_-_2bin
Starting: BL-CAM_LIGHT_-_2016-12-02_-_RiLA600_STX-16803_-_2bin
Starting: BL-CAM_LIGHT_-_2016-12-05_-_RiLA600_STX-16803_-_2bin
Starting: BL-CAM_LIGHT_-_2016-12-07_-_RiLA600_STX-16803_-_2bin
Starting: BL-CAM_LIGHT_-_2016-12-09_-_RiLA600_STX-16803

In [137]:
# for DOINGDIR in DOINGDIRs[:] : 
#     DOINGDIR = Path(DOINGDIR)
#     print(f"Starting: {str(DOINGDIR.parts[-1])}")
#     fits_in_dir = sorted(list(DOINGDIR.glob('*.fit*')))
#     #print("fits_in_dir", fits_in_dir)
#     print("len(fits_in_dir)", len(fits_in_dir))

#     if len(fits_in_dir) == 0 :
#         print(f"There is no fits fils in {DOINGDIR}")
#         pass
#     else : 
#         summary = None 
#         summary = yfu.make_summary(DOINGDIR/"*.fit*",
#                     #output = save_fpath,
#                     verbose = False
#                     )
#         print("summary: ", summary)
#         print("len(summary)", len(summary))

In [138]:
# for _, row in summary.iterrows():
#     fpath = Path(row["file"])
#     print (f"starting {fpath.name}...")
#     if fpath.suffix == ".fit" and \
#         (fpath.parents[0]/f"{fpath.stem}.fits").exists() : 
#         os.remove(fpath)
#         print(f"{fpath.name} is removed")

In [139]:
# for DOINGDIR in DOINGDIRs[:] : 
#     DOINGDIR = Path(DOINGDIR)
#     print(f"Starting: {str(DOINGDIR.parts[-1])}")
#     fits_in_dir = sorted(list(DOINGDIR.glob('*.fit*')))
#     #print("fits_in_dir", fits_in_dir)
#     print("len(fits_in_dir)", len(fits_in_dir))

#     if len(fits_in_dir) == 0 :
#         print(f"There is no fits fils in {DOINGDIR}")
#         pass
#     else : 
#         summary = None 
#         summary = yfu.make_summary(DOINGDIR/"*.fit*",
#                     #output = save_fpath,
#                     verbose = False
#                     )
#         print("summary: ", summary)
#         print("len(summary)", len(summary))

#         for _, row in summary.iterrows():
#             fpath = Path(row["file"])
#             print (f"starting {fpath.name}...")
#             if {fpath.suffix} == ".fit" and \
#                 (fpath.parents[0]/f"{fpath.stem}.fits").exists() : 
#                 os.remove(fpath)
#                 print(f"{fpath.name} is removed")

In [140]:
# fpath.suffix
# fpath.parent